In [26]:
import json
import torch
import torch.nn as nn
import numpy as np
import torch.nn as NN 
from torch.utils.data import Dataset, DataLoader
from nltk_utils import tokenize, stem, bag_of_words

with open('intents.json', 'r') as f:
    intents = json.load(f)


device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [27]:
all_words =[]
tags=[]
xy=[]
for intent in intents['intents']:
    tag= intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w=tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag, ))
ignore_words = ['?', '!', '.', ',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))
print(tags)
print(xy)

['Buy', 'Car Brand', 'Car Type', 'Goodbye', 'Greeting', 'Noanswer', 'Purchasing', 'Thanks']
[(['Hi'], 'Greeting'), (['How', 'are', 'you', '?'], 'Greeting'), (['Is', 'anymore', 'there', '?'], 'Greeting'), (['Hey'], 'Greeting'), (['Good', 'day'], 'Greeting'), (['I', 'want', 'to', 'buy', 'a', 'car'], 'Buy'), (['I', 'am', 'looking', 'for', 'a', 'car'], 'Buy'), (['I', 'need', 'a', 'car'], 'Buy'), (['car'], 'Buy'), (['Jeep'], 'Buy'), (['New'], 'Car Type'), (['Second', 'hand'], 'Car Type'), (['Branded'], 'Car Type'), (['old'], 'Car Type'), (['x'], 'Car Brand'), (['Y'], 'Car Brand'), (['Z'], 'Car Brand'), (['ABC'], 'Purchasing'), (['XYZ'], 'Purchasing'), (['PQR'], 'Purchasing'), (['A'], 'Purchasing'), (['B'], 'Purchasing'), (['c'], 'Purchasing'), (['XY'], 'Purchasing'), (['YZ'], 'Purchasing'), (['ZA'], 'Purchasing'), (['PQ'], 'Purchasing'), (['QR'], 'Purchasing'), (['RA'], 'Purchasing'), (['Bye'], 'Goodbye'), (['Thanks', 'for', 'visiting', 'our', 'site'], 'Goodbye'), (['See', 'you', 'later'], 

In [28]:

X_train =[]
Y_train =[]
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    Y_train.append(label)

X_train =np.array(X_train)
Y_train=np.array(Y_train)





print(X_train)
print(Y_train)




[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[4 4 4 4 4 0 0 0 0 0 2 2 2 2 1 1 1 6 6 6 6 6 6 6 6 6 6 6 6 3 3 3 3 3 3 7 7
 7 7 7]


In [29]:

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples= len(X_train)
        self.x_data=X_train
        self.y_data=Y_train
  
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

    
    

In [42]:


dataset= ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)



In [43]:

output_size=len(tags)
input_size=len(X_train[0])
print(input_size, len(all_words))
print(output_size, tags)

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet,self).__init__()
        self.l1=nn.Linear(input_size, hidden_size)
        self.l2=nn.Linear(hidden_size, hidden_size)
        self.l3=nn.Linear(hidden_size, num_classes)

        self.relu=nn.ReLU()

    def forward(self,x ):
        out =self.l1(x)
        out=self.relu(out)
        out=self.l2(out)
        out=self.relu(out)
        out =self.l3(out)
        out=self.relu(out)

        return out

    



58 58
8 ['Buy', 'Car Brand', 'Car Type', 'Goodbye', 'Greeting', 'Noanswer', 'Purchasing', 'Thanks']


In [62]:

model =NeuralNet(input_size, hidden_size, output_size).to(device)

num_epochs=1300
hidden_size=5
lr=0.001
batch_size=8

criterion =nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(), lr=lr)
for epoch in range (num_epochs):
    for (words, labels) in train_loader:
        
        words= words.to(device)
        labels= labels.to(dtype=torch.long).to(device)
        optimizer.zero_grad()

        outputs=model(words)
        loss = criterion(outputs, labels)

      
        loss.backward()
        optimizer.step()
    if (epoch +1) %100 ==0:
     print(f'epoch{epoch+1}/{num_epochs}, loss={loss.item():.4f}')   
    
print(f'final loss, loss={loss.item():.4f}')   

epoch100/1300, loss=1.4248
epoch200/1300, loss=1.0136
epoch300/1300, loss=1.4536
epoch400/1300, loss=0.1091
epoch500/1300, loss=0.0267
epoch600/1300, loss=1.0504
epoch700/1300, loss=0.5244
epoch800/1300, loss=1.0475
epoch900/1300, loss=0.5234
epoch1000/1300, loss=0.5210
epoch1100/1300, loss=1.0416
epoch1200/1300, loss=0.5211
epoch1300/1300, loss=0.5212
final loss, loss=0.5212


In [63]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth
